<h1><center>Laboratorio 5: Benchmark Bayesiano 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Diego Irarrázaval

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Joaquín Roa
- Nombre de alumno 2: Matías Sepúlveda


### **Link de repositorio de GitHub:**  https://github.com/jroallanos/LDPCPCDD19

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Objetivos principales del laboratorio](#Objetivos-principales-del-laboratorio)

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 4/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Obtener datos desde Reddit y visualizar cuales post son más probables que sean puntuados positivamente.
- Aplicar un atajo bayesiano para obtener la mean posterior de datos.
- Optimizar a través de librerías funciones.

#Importamos librerias utiles 😸

In [1]:
%%capture
!pip install praw
!pip install line_profiler
import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

In [2]:
!pip install memory_profiler

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://styles.redditmedia.com/t5_2rer8/styles/bannerBackgroundImage_6o2td1zc54671.jpg?width=4000&format=pjpg&s=600bcf8560dff264f1cc7c0785ba5f6d529e0c28" width="10000">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema Bayesiano, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a las subfunciones ser ejecutadas (por linea de código, vean las clases). Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del Código a través de un perfilador.

In [3]:
%%file praw_module.py
import praw
import numpy as np
def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
  reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
                     client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
                     password='ClasesMDS7202',
                     user_agent='Clases',
                     username='DocenciaDataScience', check_for_async=False)

  subreddit  = reddit.subreddit(nombre_subreddit)

  votes, post, url = {}, {}, {}
  top_submissions = list(subreddit.hot(limit = n_hot))
  for it, top_n in enumerate(range(50, len(top_submissions), 50)):
    top_n_submissions = top_submissions[:top_n]
    upvotes, downvotes, url[it], post[it] = [], [], [], []

    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
  return votes, post, url

Overwriting praw_module.py


In [3]:
def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
  reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
                     client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
                     password='ClasesMDS7202',
                     user_agent='Clases',
                     username='DocenciaDataScience', check_for_async=False)

  subreddit  = reddit.subreddit(nombre_subreddit)

  votes, post, url = {}, {}, {}
  top_submissions = list(subreddit.hot(limit = n_hot))
  for it, top_n in enumerate(range(50, len(top_submissions), 50)):
    top_n_submissions = top_submissions[:top_n]
    upvotes, downvotes, url[it], post[it] = [], [], [], []

    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
  return votes, post, url

In [5]:
%prun praw_reddit(nombre_subreddit = 'chile', n_hot = 1000)

Se agregarán solamente los primeros debido a la gran extensión de resultados:
```
363799 function calls (248686 primitive calls) in 17.485 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      273   16.655    0.061   16.655    0.061 {method 'read' of '_ssl._SSLSocket' objects}
        2    0.145    0.072    0.145    0.072 {method 'do_handshake' of '_ssl._SSLSocket' objects}
       11    0.110    0.010    0.110    0.010 decoder.py:343(raw_decode)
        2    0.104    0.052    0.104    0.052 {method 'connect' of '_socket.socket' objects}
   111936    0.084    0.000    0.101    0.000 submission.py:549(__setattr__)
   
```

In [6]:
%load_ext line_profiler

In [7]:
%lprun -f praw_reddit praw_reddit()

```
Total time: 16.7802 s
File: /Users/matiassepulveda/Documents/Semestre Otoño 2021/Laboratorio de Ciencia de Datos/Laboratorio 5/praw_module.py
Function: praw_reddit at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     3                                           def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
     4         2        579.0    289.5      0.0    reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
     5         1          1.0      1.0      0.0                       client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
     6         1          1.0      1.0      0.0                       password='ClasesMDS7202',
     7         1          1.0      1.0      0.0                       user_agent='Clases',
     8         1          1.0      1.0      0.0                       username='DocenciaDataScience', check_for_async=False)
     9                                           
    10         1         31.0     31.0      0.0    subreddit  = reddit.subreddit(nombre_subreddit)
    11                                           
    12         1          2.0      2.0      0.0    votes, post, url = {}, {}, {}
    13         1   16718435.0 16718435.0     99.6    top_submissions = list(subreddit.hot(limit = n_hot))
    14        20         23.0      1.1      0.0    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
    15        19         94.0      4.9      0.0      top_n_submissions = top_submissions[:top_n]
    16        19         97.0      5.1      0.0      upvotes, downvotes, url[it], post[it] = [], [], [], []
    17                                           
    18      9519       5571.0      0.6      0.0      for submission in top_n_submissions:
    19      9500       5312.0      0.6      0.0          try:
    20      9500       7496.0      0.8      0.0              ratio = submission.upvote_ratio
    21      9500      11576.0      1.2      0.1              ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
    22      9500       6551.0      0.7      0.0              upvotes.append(ups)
    23      9500       7122.0      0.7      0.0              downvotes.append(ups - submission.score)
    24      9500       7879.0      0.8      0.0              post[it].append(submission.title)
    25      9500       8003.0      0.8      0.0              url[it].append(submission.url)
    26                                                   except Exception as e:
    27                                                       continue
    28        19       1384.0     72.8      0.0      votes[it] = np.array([upvotes, downvotes]).T
    29         1          1.0      1.0      0.0    return votes, post, url
```


In [8]:
%load_ext memory_profiler

In [9]:
%memit praw_reddit(nombre_subreddit = 'chile', n_hot = 1000)

peak memory: 155.87 MiB, increment: 2.85 MiB


In [10]:
from praw_module import praw_reddit
%mprun -f praw_reddit praw_reddit()

```
Line #    Mem usage    Increment  Occurences   Line Contents
============================================================
     3     42.8 MiB     42.8 MiB           1   def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
     4     44.8 MiB      1.9 MiB           2     reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
     5     42.9 MiB      0.0 MiB           1                        client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
     6     42.9 MiB      0.0 MiB           1                        password='ClasesMDS7202',
     7     42.9 MiB      0.0 MiB           1                        user_agent='Clases',
     8     42.9 MiB      0.0 MiB           1                        username='DocenciaDataScience', check_for_async=False)
     9                                         
    10     45.0 MiB      0.2 MiB           1     subreddit  = reddit.subreddit(nombre_subreddit)
    11                                         
    12     45.0 MiB      0.0 MiB           1     votes, post, url = {}, {}, {}
    13    135.9 MiB     90.9 MiB           1     top_submissions = list(subreddit.hot(limit = n_hot))
    14    136.0 MiB      0.0 MiB          20     for it, top_n in enumerate(range(50, len(top_submissions), 50)):
    15    136.0 MiB      0.0 MiB          19       top_n_submissions = top_submissions[:top_n]
    16    136.0 MiB      0.0 MiB          19       upvotes, downvotes, url[it], post[it] = [], [], [], []
    17                                         
    18    136.0 MiB      0.0 MiB        9519       for submission in top_n_submissions:
    19    136.0 MiB      0.0 MiB        9500           try:
    20    136.0 MiB      0.0 MiB        9500               ratio = submission.upvote_ratio
    21    136.0 MiB      0.0 MiB        9500               ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
    22    136.0 MiB      0.0 MiB        9500               upvotes.append(ups)
    23    136.0 MiB      0.0 MiB        9500               downvotes.append(ups - submission.score)
    24    136.0 MiB      0.0 MiB        9500               post[it].append(submission.title)
    25    136.0 MiB      0.0 MiB        9500               url[it].append(submission.url)
    26                                                 except Exception as e:
    27                                                     continue
    28    136.0 MiB      0.0 MiB          19       votes[it] = np.array([upvotes, downvotes]).T
    29    136.0 MiB      0.0 MiB           1     return votes, post, url
    
```


**Respuesta:**

Es posible ver que el 99,6% del tiempo de ejecución de la función se utiliza en la línea " top_submissions = list(subreddit.hot(limit = n_hot))". La siguiente mayor fracción de tiempo se emplea en la recurrencia "for submission...", con cada línea ocupando un 0,1% del tiempo total, por tanto, casi despreciable. Esta también es la línea de código que ocupa la mayor cantidad de memoria.

## 1.2 Análisis de Tiempo con Cache

Sin duda, un factor clave en la mejora del tiempo de ejecución de una aplicación es el uso eficiente de la memoria. Por lo que es importante, que ustedes respondan las siguientes preguntas:
1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder preguntas.
- [ ] Mejorar el código con cache.


**Respuestas Teóricas:**

1. La memoria cache es un tipo de memoria volátil y muy rápida, esto ya que el procesador tiene acceso directo, casi instantáneo, y es donde se almacenan los datos que más se utilizan para tenerlos de manera inmediata. LRU o Least Recently Used, es un algoritmo que descarta primero los elementos menos utilizados recientemente de la memoria cache.

2. Aumentar el consumo de memoria, si dicha memoria está alojada en el disco, entonces el acceso puede ser muy lento y así el rendimiento decaer drásticamente.

3. Se podría reducir drásticamente el uso de memoria cuando guardamos el top_submissions, y de esta manera nuestra función se ejecutaría mucho más rápido.

In [4]:
@lru_cache
def praw_reddit_cache(nombre_subreddit = 'chile', n_hot = 1000):
  reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
                     client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
                     password='ClasesMDS7202',
                     user_agent='Clases',
                     username='DocenciaDataScience', check_for_async=False)

  subreddit  = reddit.subreddit(nombre_subreddit)

  votes, post, url = {}, {}, {}
  top_submissions = list(subreddit.hot(limit = n_hot))
  for it, top_n in enumerate(range(50, len(top_submissions), 50)):
    top_n_submissions = top_submissions[:top_n]
    upvotes, downvotes, url[it], post[it] = [], [], [], []

    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
  return votes, post, url

In [5]:
%%timeit
praw_reddit()

KeyboardInterrupt: 

In [14]:
%%timeit
praw_reddit_cache()

53.2 ns ± 1.82 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## 1.3 Obtención de Mean Posterior y Standard Error

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia Bayesiana. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$
$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [6]:
votes, post, url = praw_reddit_cache()

In [7]:
def intevalos(votos_positivos, votos_negativos):
    mu_final = []
    sigma_final = []
    for index in range(len(votos_positivos)):
        a = 1 + votos_positivos[index]
        b = 1 + votos_negativos[index]
        sigma = 1.65 * (a*b/((a+b)**2)*(a+b+1))**0.5
        mu = a/(a+b)
        mu_final.append(mu)
        sigma_final.append(sigma)
    return mu_final, sigma_final
    
    
def intevalos_numpy(votos_positivos, votos_negativos):
    a = 1 + votos_positivos
    b = 1 + votos_negativos
    sigma = 1.65 * np.sqrt(a*b/((a+b)**2)*(a+b+1))
    mu = a/(a+b)
    return mu, sigma

In [98]:
%%timeit
intevalos(votes[1][:,0], votes[1][:,1]);

234 µs ± 35.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [99]:
%%timeit
intevalos_numpy(votes[1][:,0], votes[1][:,1]);

11.3 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## 1.4 Comparación de rendimiento

Creadas las funciones de la sección 1.3, de forma iterativa compare los diferentes batches de datos que contiene el diccionario `votes`. Para esto genere un gráfico utilizando plotly, donde se pueda observar las diferencias de tiempo que toma ejecutar las diferentes cantidades de datos. ¿es posible observar una diferencia? ¿a qué se deberá esto?.

Aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. Grafique estos tiempos y observe comente los desempeños.

In [8]:
@jit(nopython=True)

def intevalos_JIT(votos_positivos, votos_negativos):
    mu_final = []
    sigma_final = []
    for index in range(len(votos_positivos)):
        a = 1 + votos_positivos[index]
        b = 1 + votos_negativos[index]
        sigma = 1.65 * (a*b/((a+b)**2)*(a+b+1))**0.5
        mu = a/(a+b)
        mu_final.append(mu)
        sigma_final.append(sigma)
    return mu_final, sigma_final

In [9]:
@jit(nopython=True)
def intevalos_numpy_JIT(votos_positivos, votos_negativos):
    a = 1 + votos_positivos
    b = 1 + votos_negativos
    sigma = 1.65 * np.sqrt(a*b/((a+b)**2)*(a+b+1))
    mu = a/(a+b)
    return mu, sigma

In [32]:
cant_datos = []
for i in range(len(votes)):
    cant_datos.append(len(votes[i]))

In [33]:
time_intevalos = []
for i in range(len(votes)):
    inicio = time.time()
    intevalos(votes[i][:,0], votes[i][:, 1]);
    final = time.time()
    time_intevalos.append(final-inicio)
    
time_intevalos_JIT = []
for i in range(len(votes)):
    inicio = time.time()
    intevalos_JIT(votes[i][:,0], votes[i][:, 1]);
    final = time.time()
    time_intevalos_JIT.append(final-inicio)

In [34]:
time_numpy_intevalos = []
for i in range(len(votes)):
    inicio = time.time()
    intevalos_numpy(votes[i][:,0], votes[i][:, 1]);
    final = time.time()
    time_numpy_intevalos.append(final-inicio)
    
time_numpy_intevalos_JIT = []
for i in range(len(votes)):
    inicio = time.time()
    intevalos_numpy_JIT(votes[i][:,0], votes[i][:, 1]);
    final = time.time()
    time_numpy_intevalos_JIT.append(final-inicio)

In [35]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x = cant_datos, y = time_intevalos,
                    mode='lines+markers',
                    name='intevalos'))
fig.add_trace(go.Scatter(x=cant_datos, y = time_intevalos_JIT,
                    mode='lines+markers',
                    name='intevalos_JIT'))

fig.add_trace(go.Scatter(x = cant_datos, y = time_numpy_intevalos,
                    mode='lines+markers',
                    name='intevalos_numpy'))
fig.add_trace(go.Scatter(x=cant_datos, y = time_numpy_intevalos_JIT,
                    mode='lines+markers',
                    name='intevalos_numpy_JIT'))

fig.update_layout(title='Time vs size_data',
                   xaxis_title = 'Cantidad de datos',
                   yaxis_title='Tiempo de ejecución')

fig.show()

Se puede apreciar en el gráfico que la función que tiene los mayores tiempos es la de "intevalos()", que tiene la recursión y no utiliza Numpy ni JIT. Así también, podemos apreciar que la función más rápida es la que utiliza Numpy y Jit, lo cual era esperable dado lo visto en clases. No existe una tendencia apreciable en las dos funciones restantes.


## 1.5 Plot de Resultados Bayesianos

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [26]:
votos = votes[len(votes)-1]
posteo = post[len(post)-1]
print("lower bounds aproximados:")
posterior_mean, std_err = intevalos_numpy(votos[:, 0], votos[:, 1])
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {'Votos (+)':votos[order[:20], 0], 'Votos (-)':votos[order[:20], 1] ,'Post':np.array(posteo)[order[:20]], 'url': np.array(url[len(url)-1])[order[:20]]}
df = pd.DataFrame(vote_post)
ordered_post = df.Post
df


lower bounds aproximados:

Top 20 post ordenador por el limite inferior:



,Votos (+),Votos (-),Post,url
0,930,0,Gato andino extremadmente raro mirando una cám...,https://v.redd.it/k2r2bhlh0h971
1,719,0,"Debate presidencial 2005, pregunta clave de Ma...",https://v.redd.it/xj4g8rc5sw671
2,522,0,¿Qué hacer con Talca? Bueno...,https://i.redd.it/1tk589eq89771.png
3,455,0,"El bombardero de Lo Prado: ""Se lo mando seco e...",https://v.redd.it/l6c86bhd2w571
4,441,0,"Aysén hoy 2, no se subieron todas juntas (Bonu...",https://www.reddit.com/gallery/o6f7lr
5,402,0,"Juan Carlos Bodoque, This Charming Man (Video ...",https://v.redd.it/wf0yqt92tv771
6,357,0,Fotografía de un vendedor ambulante en la Plaz...,https://www.reddit.com/gallery/oc0qzt
7,342,0,Avión Pingüino Rey de Aerovías DAP que vuela e...,https://i.redd.it/f82si2h743771.jpg
8,271,0,Cuando el chico Paris y Siches eran compañeros...,https://v.redd.it/nkfiwmwgma571
9,252,0,Con 99% de aprobación: Bomberos es la instituc...,https://www.24horas.cl/nacional/bomberos-aprob...


In [27]:
r_order = order[::-1][-20:]
data_dic = {'mean':posterior_mean[r_order], 'std_err':std_err[r_order], 'post':ordered_post[::-1]}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post", error_x="std_err")
fig.show()

**Respuesta:**

Logramos observar en los resultados que el algoritmo prioriza los post que tienen una mayor relación de votos positivos sobre negativos. Esta solución no es trivial, ya que, muchas publicaciones con muchos votos positivos resulta con una mayor ponderación por tener algunos votos negativos.

Además, es curioso ver los post que se encuentran en el top, ya que podemos ver una gran variedad de tipos, desde temas netamente "divertidos" o de broma, hasta otros relacionados a tópicos poco controversiales, como nuestro amor por bomberos, que es transversal a todo ciudadano. 

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>